In [1]:
import perceval as pcvl
from perceval.rendering.circuit import SymbSkin, PhysSkin
from typing import Union
from perceval.components.unitary_components import Unitary, BS, PS, StateVector, BasicState
import numpy as np
from tqdm import tqdm
#from tensorboard import summary as summary_lib
import scipy

In [2]:
# Input: parameters of the discriminator, starting index of first node, name of discrim
# Output: discriminator
def create_discriminator(discriminator_params, start_mode, called):
    pses = []
    for i in range(6):
        ps = PS(phi=pcvl.P(called + f'{i}'))
        ps.assign({called + f'{i}':discriminator_params[i]})
        pses.append(ps)
        
    
    discriminator = pcvl.Circuit(4, name=called)
    discriminator.add(start_mode, pses[0])
    discriminator.add(start_mode+1, pses[1])
    discriminator.add(start_mode+2, pses[2])
    discriminator.add(start_mode+0, BS())
    discriminator.add(start_mode+2, BS())
    discriminator.add(start_mode, pses[3])
    discriminator.add(start_mode+2, pses[4])
    discriminator.add(start_mode, BS())
    discriminator.add(start_mode+2, BS())
    discriminator.add(start_mode+1, BS())
    discriminator.add(start_mode+1, pses[5])
    discriminator.add(start_mode+1, BS())
    
    return discriminator

# Input: discriminator and new parameters
# What it does: changes directly value of discriminator
def set_params_discriminator(discriminator, new_params):
    params = discriminator.get_parameters()
    for i in range(6):
        params[i].set_value(new_params[i])

In [3]:
def create_generator(generator_params, start_mode, called):
    quantum_circuit = pcvl.Circuit(4, name=called)
    
    pses = []
    for i in range(15):
        ps = PS(phi=pcvl.P(called + f'{i}'))
        ps.assign({called + f'{i}':generator_params[i]})
        pses.append(ps)
    
    quantum_circuit.add(start_mode + 1, pses[0])
    quantum_circuit.add(start_mode + 2, pses[1])
    quantum_circuit.add(start_mode + 3, pses[2])

    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 1, pses[7])
    quantum_circuit.add(start_mode + 1, BS())

    # env1
    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode, pses[3])
    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode+1, pses[4])

    quantum_circuit.add(start_mode + 2, BS())
    quantum_circuit.add(start_mode + 2, pses[8])
    quantum_circuit.add(start_mode + 2, BS())

    # env2
    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 1, pses[5])
    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 2, pses[6])

    quantum_circuit.add(start_mode, pses[9])

    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode + 2, BS())


    quantum_circuit.add(start_mode, pses[10])
    quantum_circuit.add(start_mode + 2, pses[11])

    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode + 2, BS())

    quantum_circuit.add(start_mode, pses[12])
    quantum_circuit.add(start_mode + 1, pses[13])
    quantum_circuit.add(start_mode + 2, pses[14])
    
    return quantum_circuit
    
def set_params_generator(generator, new_params):
    params = generator.get_parameters()
    for i in range(15):
        params[i].set_value(new_params[i])    

In [4]:
## Initialize our circuit
DISCRIM_SIZE = 6
GEN_SIZE = 15

disc_init1 = np.random.randint(0,2,size=(DISCRIM_SIZE))
disc_init2 = np.random.randint(0,2,size=(DISCRIM_SIZE))
gen_init1 = np.random.randint(0,2,size=(GEN_SIZE))
gen_init2 = np.random.randint(0,2,size=(GEN_SIZE))
discrim1 = create_discriminator(disc_init1,0,'d_0')
discrim2 = create_discriminator(disc_init2,0,'d_1')
gen1 = create_generator(gen_init1,0,'g_0')
gen2 = create_generator(gen_init2,0,'g_1')

tdiscrim = pcvl.Circuit(8)
tdiscrim.add(0, discrim1)
tdiscrim.add(4, discrim2)

tgen = pcvl.Circuit(8)
tgen.add(0, gen1)
tgen.add(4, gen2)

In [12]:
##Backprop 

class Backprop():
    def __init__(self, gen, discrim1, discrim2):
        self.gen = gen
        self.discrim1 = discrim1
        self.discrim2 = discrim2
        self.discrim = pcvl.Circuit(8)
        self.discrim.add(discrim1, 0)
        self.discrim.add(discrim2, 0)
        self.discrim_init = np.array([param.evalf() for param in discrim.get_parameters()])
        self.gen_init = np.array([param.evalf() for param in gen.get_parameters()])
        self.simulator = pcvl.Simulator(pcvl.NaiveBackend())
        self.tau_sv = StateVector("|1,0,0,0,0,1,0,0>") + StateVector("|0,1,0,0,0,0,1,0>") + StateVector("|0,0,1,0,0,0,0,1>") + StateVector("|0,0,0,1,1,0,0,0>")
        self.simulator.set_circuit(self.gen)
        self.initial_sv = StateVector("|1,0,0,0,1,0,0,0>") + StateVector("|0,1,0,0,0,1,0,0>") + StateVector("|0,0,1,0,0,0,1,0>") + StateVector("|0,0,0,1,0,0,0,1>")
        
    def get_density_matrix(self,sv):
        basic_states_8 = [BasicState("|1,0,0,0,1,0,0,0>"), # |00>
                        BasicState("|1,0,0,0,0,1,0,0>"), # |01>
                        BasicState("|1,0,0,0,0,0,1,0>"), # |02>
                        BasicState("|1,0,0,0,0,0,0,1>"), # |03>
                        BasicState("|0,1,0,0,1,0,0,0>"), # |10>
                        BasicState("|0,1,0,0,0,1,0,0>"), # |11>
                        BasicState("|0,1,0,0,0,0,1,0>"), # |12>
                        BasicState("|0,1,0,0,0,0,0,1>"), # |13>
                        BasicState("|0,0,1,0,1,0,0,0>"), # |20>
                        BasicState("|0,0,1,0,0,1,0,0>"), # |21>
                        BasicState("|0,0,1,0,0,0,1,0>"), # |22>
                        BasicState("|0,0,1,0,0,0,0,1>"), # |23>
                        BasicState("|0,0,0,1,1,0,0,0>"), # |30>
                        BasicState("|0,0,0,1,0,1,0,0>"), # |31>
                        BasicState("|0,0,0,1,0,0,1,0>"), # |32>
                        BasicState("|0,0,0,1,0,0,0,1>"), # |33>
                        ]
        sv_amplitudes = np.array([])
        for state in basic_states_8:
            sv_amplitudes = np.append(sv_amplitudes, sv[state])
        sv_amplitudes_bra = np.conjugate(sv_amplitudes.T)
        rho = np.outer(sv_amplitudes, sv_amplitudes_bra)
        return rho


    def make_mats(self):    
        tau = self.get_density_matrix(self.tau_sv)
        return tau

    def D(self,psi_G):
        rho = self.get_density_matrix(psi_G)
        # print(rho)
        p_s = pcvl.Matrix(discrim1.U)
        p_i = pcvl.Matrix(discrim2.U)
        M_hat = np.kron(p_s, p_i)

        expectation_value_rho = np.trace(M_hat @ rho)
        expectation_value_tau = np.trace(M_hat @ self.make_mats())
        return np.abs(expectation_value_rho - expectation_value_tau)            
    
    def prop_gen(self, params_list):
        dist_i = self.D(self.simulator.evolve(self.initial_sv))

        for i,param in enumerate(self.gen.get_parameters()):
            param.set_value(params_list[i])
        self.simulator.set_circuit(self.gen)
        dist_f = self.D(self.simulator.evolve(self.initial_sv))
        #print(dist_i,dist_f)
        print(f"G:{dist_f}")
        return dist_f
    
    def prop_discrim(self, params_list):
        dist_i = self.D(self.simulator.evolve(self.initial_sv))

        for i,param in enumerate(self.discrim.get_parameters()):
            param.set_value(params_list[i])
            if(i<6):
                discrim1.get_parameters()[i].set_value(params_list[i])
            else:
                discrim2.get_parameters()[i].set_value(params_list[i])
            
        self.simulator.set_circuit(self.discrim)
        dist_f = self.D(self.simulator.evolve(self.initial_sv))
        #print(dist_i,dist_f)
        print(f"D:{dist_f}")
        return -dist_f
    
    def run_gen_optim(self):
        options = {'maxiter': 5}
        out=scipy.optimize.minimize(self.prop_gen, self.gen_init, options=options)
        print(f"G:{out}")
        
    def run_discrim_optim(self):
        options = {'maxiter': 5}
        out=scipy.optimize.minimize(self.prop_discrim, self.discrim_init,options=options)
        print(f"D:{out}")
    
    def mog(self, epochs):
        for epoch in tqdm(range(epochs)):
            self.run_gen_optim()
            self.run_discrim_optim()
            self.run_gen_optim()
        return self.gen

            
Back = Backprop(tgen,discrim1,discrim2)

gen = Back.mog(1)


simulator = pcvl.Simulator(pcvl.NaiveBackend())
simulator.set_circuit(gen)
print(simulator.evolve(StateVector("|1,0,0,0,1,0,0,0>") + StateVector("|0,1,0,0,0,1,0,0>") + StateVector("|0,0,1,0,0,0,1,0>") + StateVector("|0,0,0,1,0,0,0,1>")))

AssertionError: Only unitary components can compose a linear optics circuit, use Processor for non-unitary